In [1]:
import sys 
sys.path.append(r"C:\Users\a-sochat\development\Qcodes_Sohail_Clone\Qcodes")

import numpy as np

import qcodes as qc
from qcodes.instrument.parameter import ManualParameter
from qcodes.sweep.sweep import Nest, Zip, Chain, ParameterSweep, ParameterWrapper, FunctionSweep, FunctionWrapper

# We will first make a print function which conviniently prints the output of our sweep objects. 

In [2]:
def pretty_print(dictionary): 
    
    s = ""
    independents = ""
    seperator = ""
    
    names = list(dictionary.keys())[::-1]
    
    for name in names: 
        value = str(dictionary[name]["value"])
        
        if dictionary[name]["independent_parameter"]: 
            s += seperator + name + value
            independents = s
        else: 
            s += seperator + name + "({}) = {}".format(independents, value) 
        
        seperator = ", "
    
    print(s)

In [3]:
# At each iteration a sweep object will return a dictionary in the form...

d = {"x": {"unit": "", "value": 3.4, "independent_parameter": True}}
pretty_print(d)

x3.4


independent parameters will be printed as "name value"

In [4]:
d = {"x": {"unit": "", "value": 3.4, "independent_parameter": False}}
pretty_print(d)

x() = 3.4


dependent parameters will be printed as name() = value.

In [5]:
d = {
    "y": {"unit": "", "value": 1.2, "independent_parameter": False}, 
    "x": {"unit": "", "value": 3.4, "independent_parameter": True}
}
pretty_print(d)

x3.4, y(x3.4) = 1.2


The independent parameter x has value 3.4. The dependent parameter y is evaluated for x = 3.4 and has value 1.2

In [6]:
d = {
    "y": {"unit": "", "value": 1.2, "independent_parameter": True}, 
    "x": {"unit": "", "value": 3.4, "independent_parameter": True}
}
pretty_print(d)

x3.4, y1.2


# Lets introduce a basic sweep object 

In [7]:
x = ManualParameter("x")

In [8]:
for i in ParameterSweep(x, lambda: [0, 1]): 
    print(i)

{'x': {'unit': '', 'value': 0, 'independent_parameter': True}}
{'x': {'unit': '', 'value': 1, 'independent_parameter': True}}


Lets use pretty print to make the results more readable 

In [9]:
for i in ParameterSweep(x, lambda: [0, 1]): 
    pretty_print(i)

x0
x1


We have generated a small 1D coordinate layout with size 2 

# How do we make a 2D sweep? 

In [10]:
y = ManualParameter("y")

In [11]:
sox = ParameterSweep(x, lambda: [0, 1])
soy = ParameterSweep(y, lambda: [0, 1])

In [12]:
for i in Nest([soy, sox]):  # X is the inner axis : 
    pretty_print(i)

y0, x0
y0, x1
y1, x0
y1, x1


This represents a 2D layout of 2x2 

# We can extend this to ND

In [13]:
z = ManualParameter("z")

In [14]:
sox = ParameterSweep(x, lambda: [0, 1])
soy = ParameterSweep(y, lambda: [0, 1])
soz = ParameterSweep(z, lambda: [0, 1])

In [15]:
for i in Nest([soz, soy, sox]): 
    pretty_print(i)

z0, y0, x0
z0, y0, x1
z0, y1, x0
z0, y1, x1
z1, y0, x0
z1, y0, x1
z1, y1, x0
z1, y1, x1


We have created a 2x2x2 layout 

# This is how we can perform a measurement  

In [16]:
m = ManualParameter("m")
m.get = lambda: "{:.3}".format(np.random.uniform(0, 1))

In [17]:
for i in ParameterWrapper(m): 
    pretty_print(i)

m() = 0.666


Wrapping a parameter in the "ParameterWrapper" class makes a sweep object which iterates once and returns the "get" value of the parameter. We can use this to create looped measurements. 

In [18]:
sox = ParameterSweep(x, lambda: [0, 1])
soy = ParameterSweep(y, lambda: [0, 1])
measurement = ParameterWrapper(m)

In [19]:
for i in  Nest([soy, sox, measurement]): # Perform a measurement at every (x, y): 
    pretty_print(i)

y0, x0, m(y0, x0) = 0.887
y0, x1, m(y0, x1) = 0.912
y1, x0, m(y1, x0) = 0.104
y1, x1, m(y1, x1) = 0.943


The meaning of "m(x0, y0)": We measure "m" at (x0, y0). We have thus a 2x2 coordinate layout and the thrid dimension represents a measurement 

# What happens if we nest a sweep in a measurement?

In [20]:
for i in Nest([measurement, sox]): # Notice how "sox" is the inner axis
    pretty_print(i)

m() = 0.75, x0
m() = 0.75, x1


Here "m()" represents the same measurement. We thus only call "m" once but the result will be echoed in all subsequent lines in the measurement table.

# Introducing chaining

In [21]:
m = ManualParameter("m")
m.get = lambda: "{:.2}".format(np.random.uniform(0, 1))
mm = ParameterWrapper(m)

n = ManualParameter("n")
n.get = lambda: "{:.2}".format(np.random.uniform(0, 1))
nn = ParameterWrapper(n)

In [22]:
sox = ParameterSweep(x, lambda: [0, 1]) 

In [23]:
# Perform measurement1 and then iterate over the sweep
# This is approx. equivalent to m + (x * n) 
for i in Chain([mm, Nest([sox, nn])]): 
    pretty_print(i)

m() = 0.25
x0, n(x0) = 0.89
x1, n(x1) = 0.41


"m()" means that we measure m at undefined x or y. We can use the chaining mechanism to perform multiple loop simulataniously

In [24]:
sox = ParameterSweep(x, lambda: [0, 1])
soy = ParameterSweep(y, lambda: [0, 1])

In [25]:
for i in Nest([soy, Chain([mm, Nest([sox, nn])])]):  # = y * (m + x * n) = ym + yxn: 
    pretty_print(i)

y0, m(y0) = 0.56
y0, x0, n(y0, x0) = 0.039
y0, x1, n(y0, x1) = 0.79
y1, m(y1) = 0.53
y1, x0, n(y1, x0) = 0.58
y1, x1, n(y1, x1) = 0.13


We have woven together a 1D and 2D loop. Notice that we immediately see that "m" only depends on y and "n" depends on x and y. There is no need to explicitly state this. With will become important when we discuss data sets

# We will introduce a couple of conviniece functions so we do not have to be as verbose

In [26]:
from qcodes.sweep import sweep, nest, chain

In [27]:
sox = sweep(x, [0, 1])  # Notice we do not have to use lambda functions in the sweep
soy = sweep(y, [0, 1])

m = ManualParameter("m")
m.get = lambda: "{:.2}".format(np.random.uniform(0, 1))

n = ManualParameter("n")
n.get = lambda: "{:.2}".format(np.random.uniform(0, 1))

In [28]:
for i in nest(soy, chain(m, nest(sox, n))):   # We do not have to wrap parameters
    pretty_print(i)

y0, m(y0) = 0.73
y0, x0, n(y0, x0) = 0.47
y0, x1, n(y0, x1) = 0.27
y1, m(y1) = 0.011
y1, x0, n(y1, x0) = 0.36
y1, x1, n(y1, x1) = 0.93


# We can use arbitrary functions instead of parameters as measurements

In [41]:
def measurement_function(): 
    value = np.random.normal(0, 1, (3,))
    return {"measurement": {"unit": "H", "value": value, "independent_parameter": False}}

In [42]:
for i in nest(sweep(x, [0, 1, 2, 3]), measurement_function): 
    pretty_print(i)

x0, measurement(x0) = [ 0.55507601 -0.10745279  1.46508295]
x1, measurement(x1) = [ 0.12989571  0.30350761  0.36686223]
x2, measurement(x2) = [-0.10581962  0.39113143  1.33945185]
x3, measurement(x3) = [-0.33793317  0.77996067  0.1327499 ]


# We can also use functions as loop parameters

In [51]:
def setter(value): 
    # ... some code to set the independent variable
    return {"dac_channel": {"unit": "H", "value": value, "independent_parameter": True}}

In [52]:
for i in nest(sweep(setter, [0, 1, 2, 3]), measurement_function): 
    pretty_print(i)

dac_channel0, measurement(dac_channel0) = [-0.59398484  2.02110166 -1.11809521]
dac_channel1, measurement(dac_channel1) = [ 0.7642435  -0.20177533 -0.31730194]
dac_channel2, measurement(dac_channel2) = [-0.90800159  1.46225142 -0.81558136]
dac_channel3, measurement(dac_channel3) = [ 0.02866496 -0.66246136  0.30246045]


# Finally, the sweep values need not be a list or an numpy array, but also a generating function

In [82]:
class Measurement:
    def __init__(self): 
        self._value = self._roll_dice()
    def __call__(self): 
        self._value = self._roll_dice()
        return {"measurement": {"unit": "H", "value": self._value, "independent_parameter": False}}
    def _roll_dice(self): 
        return np.random.normal(0, 1, (3,))
    def value(self): 
        return self._value

measurement_function = Measurement()

# By allowing the sweep values to be a generator, we can create a feed-back loop between the sweep object and the 
# measurement.

def sweep_values(): 
    value = 0.0
    while value < 2.0:
        yield value 
        value = np.sum(measurement_function.value())  # The next step depends on the measurement

def setter(value): 
    return {"dac_channel": {"unit": "H", "value": "{:.3}".format(value), "independent_parameter": True}}

for i in nest(sweep(setter, sweep_values), measurement_function): 
    pretty_print(i)

dac_channel0.0, measurement(dac_channel0.0) = [ 1.32547921 -0.21213639  0.19645486]
dac_channel1.31, measurement(dac_channel1.31) = [-0.95784328 -0.30450607 -0.76690471]
dac_channel-2.03, measurement(dac_channel-2.03) = [-0.36953014  0.57696136 -2.53465172]
dac_channel-2.33, measurement(dac_channel-2.33) = [-1.13104468  0.55393251  0.63410781]
dac_channel0.057, measurement(dac_channel0.057) = [-1.37735186  0.78612939  0.06011491]
dac_channel-0.531, measurement(dac_channel-0.531) = [ 1.06752019 -0.46777733  0.45741794]
dac_channel1.06, measurement(dac_channel1.06) = [ -2.15684711e+00  -1.35221664e+00  -8.88756194e-04]
dac_channel-3.51, measurement(dac_channel-3.51) = [-1.07443465  0.22059757 -0.73652388]
dac_channel-1.59, measurement(dac_channel-1.59) = [ 2.5331431   1.25150278  1.6811127 ]


We will loop until the sum of the measurement variables equal 2.0 or more. Since measurement values are three stochstics with a N(0, 1) distribution, the distribution of the sum is N(0, sqrt(3)). The probability of finding 2.0 or more: Z = 2/sqrt(3), which is equal to 12.51%. As the next cell shows, the expectation value of the number of iterations is therefore 8.3 iterations

In [100]:
def f(ni): # The probability of looping exactly ni times 
    p = 0.1251
    return (1 - p)**(ni - 1) * p

np.sum([ni*f(ni) for ni in range(1, 1000)])

7.9936051159072736

Lets see if this is correct 

In [98]:
count = 0
N = 10000
for _ in range(N):
    count += len(list(nest(sweep(setter, sweep_values), measurement_function)))

print(count/N)

8.085


:-)